# 1 Mycelium Pool Simulation<a id='2_Data_wrangling'></a>

## 1.1 Contents<a id='1.1_Contents'></a>
* [1 Mycelium Pool Simulation](#1_Mycelium_Pool_Simulation)
    * [1.1 Contents](#1.1_Contents)
    * [1.2 Introduction](#1.2_Introduction)
    * [1.3 Results](#1.3_Results)
        * [2.2.1 Recap Of Data Science Problem](#2.2.1_Recap_Of_Data_Science_Problem)
        * [2.2.2 Introduction To Notebook](#2.2.2_Introduction_To_Notebook)

## 1.2 Introduction<a id='1.2_Introduction'></a>

This notebook contains a simulation of the Mycelium Pool that optimizes for minimum use of materials(calcite/pyrophosphite)

## 1.3 Results <a id='1.3_Results'></a>

The strategy that minimizes material use is to start with **7** materials. If the chemical process doesnt reach completion follow the chart below to determine how many more materials to add

$k:=$ number of materials left \\
$t:=$ ticks (or time) left until completion \\

$$\newcommand\cmat{\boldsymbol{C}}$$

We wish to find the cost matrix, $\cmat$, that describes the expected cost of reaching completion from any point in state space. \\
Its elements, $\cmat_{ij}$ are the expected cost of traveling $(k_i,t_j) → (k_{\geq3}, t_0)$. \\


We can initialize with known values (as shown below) and set `?` elements as 0.


$$
\begin{array}{c c} 
& \begin{array}{c c c c} t_0 & t_1 & t_2 & \ldots & t_n \\ \end{array} \\
\begin{array}{c c c c} k_0 \\ k_1 \\ k_2 \\ k_3 \\ k_4 \\ \vdots \\ k_m \end{array} &
\left[
\begin{array}{c c c}
0 & ? & ? & \ldots & ? \\
0 & ? & ? & \ldots & ? \\
0 & ? & ? & \ldots & ? \\
0 & 3 & ? & \ldots & ? \\
0 & 4 & ? & \ldots & ? \\
\vdots & \vdots& \vdots & \ddots & ? \\
0 & m & ? & ? & ?
\end{array}
\right]
\end{array}
$$

Then solve unknown values by propagating the expected cost formula (shown below) until convergence.

For $k\geq3:$
$$
\begin{equation}
  C(k,t) = \frac{2}{3}C(k,t-1) + \frac{1}{3}(1 + C(k-1,t-1))
\end{equation}
$$

\\

For $k < 3$ we rely on Strategy matrix A: \\

\\
$$
\begin{equation}
  C(k,t) = A[\text{argmax}_{i}, t + 2\cdot\text{strat}] + \text{strat}
\end{equation}
$$

In [1]:
import numpy as np
import pandas as pd
from IPython.display import Image

In [2]:
def init_cost_mat(shape):
    C = np.zeros(shape)
    m,n = shape
    
    C[:,0] = 0 # We're done, no cost!
    for i in range(1,n-2):
        C[i+2:,i] = np.arange(i+2,n) # These costs are fixed
    C[0:2,1:] = 999 # Unable to reach these states
    return C

In [3]:
def init_strat_mat(shape):
    A = np.zeros(shape)
    m,n = shape

    A[:,0] = 999 # Not able to access the elements where progress is zero (column1)
    A[0,:] = 999 # Not able to access the elements where materials to add is 0
    A[9:11,:] = 999 # Not able to accress the elements where materials to add is 9 or 10
    A[:,10] = 999 # Not able to have current progress == 10 and current materials == 2
    return A

In [4]:
def update_C(C,A,index):
    k,t = index
    
    if t == 0: # We're done!
        pass
    elif (k,t) in [(j,i) for i in range(1,9) for j in range(i+2,11)]: # Deterministic cost, we're done!
        pass
    elif k in (0,1): # Unable to reach these states
        pass
    elif k == 2: # We go to matrix A when decision needs to be made (k==2) (might need change, not sure if +np.argmin(A[:,t])is needed)
        C[k,t] = np.amin(A[:,t]) #+np.argmin(A[:,t]) 
    elif k >= 3: # No need to make decision, cost will use expected cost formula
        C[k,t] = (2/3)*C[k,t-1] + (1/3)*(C[k-1,t-1] + 1) # Expected cost formula
    else:
        print('Something went wrong when updating C, at least one condition should have been met')
    return C

In [5]:
def update_A(C, A, index):
    k,t = index
    
    if t == 0: # We're done!
        pass
    elif t == 10: # Unable to reach this state
        pass
    elif k in (0,9,10): # Unable to make these decisions
        pass
    elif k >= 1: # Allowed decision states
        A[k,t]= C[k+2, min(t + (2*k),10)] # Expected cost of the decision's landing state + cost of making this decision
    else:
        print('Something went wrong when updating A, at least one condition should have been met')
    return A

In [6]:
def step(C,A):
    for (k,t), val in np.ndenumerate(C):
        C=update_C(C,A,(k,t))
        A=update_A(C,A,(k,t))
    return C,A

In [7]:
def iterate(C, A, min_iters, threshold):
    sse = np.inf # Placeholder init value for sum of squared error
    num_iters = 0
    while sse > threshold or num_iters < min_iters:
        num_iters += 1
        old_C = C.copy()
        old_A = A.copy()
        C, A = step(C, A)
        sse_C = ((C - old_C)**2).sum(axis = None) # Sum of squared error of matrix C
        sse_A = ((A - old_A)**2).sum(axis = None) # Sum of squared error of matrix A
        sse = sse_C + sse_A
    return (num_iters, C, A)

In [8]:
def display(M):
    # Helper function for displaying the cost/strategy matrix
    m, n = M.shape
    df = pd.DataFrame(M, 
                  columns=[f"t{i}" for i in range(m)],
                  index=[f"k{i}" for i in range(n)])
    style = df.style \
            .highlight_min(color='green', axis=0) \
            .format(precision=2)
    return style

In [9]:
shape=(11,11)
C = init_cost_mat(shape)
A= init_strat_mat(shape)

In [10]:
C

array([[  0., 999., 999., 999., 999., 999., 999., 999., 999., 999., 999.],
       [  0., 999., 999., 999., 999., 999., 999., 999., 999., 999., 999.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   3.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   4.,   4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   5.,   5.,   5.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   6.,   6.,   6.,   6.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   7.,   7.,   7.,   7.,   7.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   8.,   8.,   8.,   8.,   8.,   8.,   0.,   0.,   0.,   0.],
       [  0.,   9.,   9.,   9.,   9.,   9.,   9.,   9.,   0.,   0.,   0.],
       [  0.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,   0.,   0.]])

In [11]:
A

array([[999., 999., 999., 999., 999., 999., 999., 999., 999., 999., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.],
       [999., 999., 999., 999., 999., 999., 999., 999., 999., 999., 999.],
       [999., 999., 999., 999., 999., 999., 999., 999., 999., 999., 999.]])

In [12]:
num_iters, C, A = iterate(C, A, 1, 0.000005)
print(f"Matrix C iterated {num_iters} times.")
display(C)

Matrix C iterated 30 times.


,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
k0,0.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00
k1,0.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00
k2,0.00,4.93,5.83,6.63,7.25,7.47,7.47,7.47,7.47,7.47,999.00
k3,0.00,3.00,3.98,4.93,5.83,6.63,7.25,7.65,7.92,8.11,8.23
k4,0.00,4.00,4.00,4.33,4.86,5.52,6.22,6.90,7.48,7.96,8.34
k5,0.00,5.00,5.00,5.00,5.11,5.36,5.74,6.24,6.79,7.35,7.89
k6,0.00,6.00,6.00,6.00,6.00,6.04,6.14,6.34,6.64,7.02,7.47
k7,0.00,7.00,7.00,7.00,7.00,7.00,7.01,7.06,7.15,7.32,7.55
k8,0.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.02,8.06,8.15
k9,0.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.01,9.03


In [13]:
print(f"Matrix A iterated {num_iters} times.")
display(A)

Matrix A iterated 30 times.


,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
k0,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00
k1,999.00,4.93,5.83,6.63,7.25,7.65,7.92,8.10,8.23,8.23,999.00
k2,999.00,5.52,6.22,6.90,7.48,7.96,8.34,8.34,8.34,8.34,999.00
k3,999.00,6.24,6.79,7.35,7.89,7.89,7.89,7.89,7.89,7.89,999.00
k4,999.00,7.02,7.47,7.47,7.47,7.47,7.47,7.47,7.47,7.47,999.00
k5,999.00,7.55,7.55,7.55,7.55,7.55,7.55,7.55,7.55,7.55,999.00
k6,999.00,8.15,8.15,8.15,8.15,8.15,8.15,8.15,8.15,8.15,999.00
k7,999.00,9.03,9.03,9.03,9.03,9.03,9.03,9.03,9.03,9.03,999.00
k8,999.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,999.00
k9,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00,999.00
